In [1]:
import nltk
import os
import torch
from transformers import pipeline
import requests
import runpod
from text_generation import Client
import json
import re
import tiktoken
nltk.download('punkt')

D:\coding\ai_learnoor\learner_env\Lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
api_key = 

In [3]:
def clean_whitespace(text):
        text = re.sub(r'\n{2,}', '\n', text)
        text = re.sub(r' {2,}', '\n', text)
        text = text.strip()
        return text

In [12]:
def load_data(data):
    with open(data, 'r', encoding='utf-8') as f:
        story = f.read()
    
    clean_story = clean_whitespace(story)
    
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = encoding.encode(clean_story)
    print(len(num_tokens))

    segments = {}
    current_segment = []
    seg_num = 1
    
    # will loop untill dict has 1000 tokens
    for token in num_tokens:
        current_segment.append(token)
        
        # if >= 1000 will add this batch to dict
        if len(current_segment) >= 1000:
            segment_text = encoding.decode(current_segment)
            segments[f'Segment: {seg_num}'] = segment_text
            current_segment = []
            seg_num += 1
            
    # grabs remaining tokens        
    if current_segment:
        segment_text = encoding.decode(current_segment)
        segments[f'Segment: {seg_num}'] = segment_text
     
    return segments

segmented_data = load_data("D:\coding\llms\sci_storys\story4.txt")

3642


In [46]:
def gen_prompt(input_text, api_key):
    url = "https://api.runpod.ai/v2/llama2-13b-chat/runsync"

    prompt= f"""
    Based on the following story segment '{input_text}', directly create a brief sci-fi story prompt. 
    Start the prompt immediately without any introduction, explanation, or additional words. 
    End the prompt without any concluding remarks or questions. Provide only the prompt, exactly as requested, nothing more, nothing less.
    """




    payload = { "input": {
        "prompt": prompt,
        "sampling_params": {
            "max_tokens": 1000,
            "n": 1,
            "best_of": None,
            "presence_penalty": 0,
            "frequency_penalty": 0.2,
            "temperature": 0.6,
            "top_p": 1,
            "top_k": -1,
            "use_beam_search": False,
            "ignore_eos": False,
            "logprobs": None
        }
    } }
    headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": api_key
    }
    
    response = requests.post(url, json=payload, headers=headers)

    response_data = response.json()

    job_id = response_data.get('id')
    return job_id  
    

def poll_for_result(request_id, api_key, interval=5):
    status_url = f"https://api.runpod.ai/v2/llama2-13b-chat/status/{request_id}"  
    headers = {
        "accept": "application/json",
        "authorization": api_key
    }

    while True:
        response = requests.get(status_url, headers=headers)
        result = response.json()

        if result['status'] == 'COMPLETED':
            output = result.get('output')
            return output  
        elif result['status'] in ['FAILED', 'ERROR']:
            print("Error or Failed Status:", result)
            return result

        time.sleep(interval)  # Wait before polling again

In [ ]:
def extract_to_prompt(segmented_data):
    ''' 
    takes segmented input (dictionary) loops through,
    "gen_prompt" & "poll_for_result" functions
    takes llm gen prompt from output
    and places in new prompt with input text
    ''' 
    i = 1
    json_objects = []
    # loop through segmented outputs
    for _, text in segmented_data.items():
        input_text = text
        job_id = gen_prompt(input_text, api_key)
        if job_id:
            output = poll_for_result(job_id, api_key)
            
            text = output['text'][0] if output['text'] else None
            extracted_text = ""
            
            # extract text between \n\n
            if text:
                matches = re.findall(r"\n\n(.*?)(?:\n\n|$)", text, re.DOTALL)
                if matches:
                    extracted_text = matches[0].strip()
                    json_obj = {
                      "messages": [
                        {"role": "system", "content": "You are the greatest sci-fi story author in the universe."},
                        {"role": "user", "content": extracted_text},
                        {"role": "assistant", "content": input_text}
                      ]
                    }
                    json_objects.append(json_obj) 
        i += 1

        return json_objects
json_output = extract_to_prompt(segmented_data)
json_output

In [51]:
text = output['text'][0] if output['text'] else None
# extract text between \n\n
if text:
    matches = re.findall(r"\n\n(.*?)(?:\n\n|$)", text, re.DOTALL)
    if matches:
        extracted_text = matches[0].strip()


# segments = extracted_text.split('\n')
# extracted_prompt = segments[2].strip() if len(segments) > 1 else None    
# extracted_prompt = extracted_prompt.strip("'")

print(extracted_text)

Lexi and Steve were on the run from an angry mob after a bizarre incident involving the Pope and a mysterious discovery. They had holed up in a run-down safe house on the outskirts of town, but they knew it was only a matter of time before the mob found them. As they huddled together on a dirty couch, watching the news in disbelief, they suddenly heard a strange noise coming from outside. It sounded like something was trying to get in.


In [101]:
json_obj = {
  "messages": [
    {"role": "system", "content": "You are the greatest sci-fi story author in the universe."},
    {"role": "user", "content": extracted_text},
    {"role": "assistant", "content": input_text}
  ]
}
json_obj

{'messages': [{'role': 'system',
   'content': 'You are the greatest sci-fi story teller the world has seen.'},
  {'role': 'user',
   'content': "What if Lexi's discovery led to a catastrophic event that threatened the fabric of reality? Could she use her knowledge of the Vacuity Machine to prevent the impending doom, or would the power of the black holes prove too great for her to control?"},
  {'role': 'assistant',
   'content': 'It was approaching 2am when Lexi made the discovery that would fracture the world. She skulled the rest of her black coffee and sat the paper cup down on the smooth, white table. She was at the university working on her PhD thesis. The research involved creating controlled, miniature black holes in a Vacuity Machine and then testing its potential for hyper-space travel.'}]}